# Inheritance and Polymorphism

Class inheritance or subclassing in python follows the basic example:

In [2]:
# base class
class User:
    is_admin = False # class variable
    
    # constructor
    def __init__(self, username):
        self.username = username # instance variable
        
# subclass
class Admin(User):
    is_admin = True

In [9]:
user = User('Tom')
admin = Admin('Mike')

print(user.username, user.is_admin)
print(admin.username, admin.is_admin)

Tom False
Mike True


Our `Admin` subclass has inherited the constructor(we need to pass it the username on initialization) and class variable, `is_admin` from the base or parent class, `User`.

We can validate that a class is a subclass through the `issubclass()` function. It takes two arguments and returns `True` if the 1st argument is a subclass of the 2nd, otherwise `False`. It raises a `TypeError` if either argument is not a class.

In [11]:
issubclass(Admin, User)

True

In [12]:
issubclass(User, Admin)

False

### Inheritance and Python Exceptions

In Python, exceptions (errors that are raised) inherit from the `Exception` class. Thus `TypeError`, `ZeroDivisionError`, etc all inherit from the `Exception` class.

In [13]:
issubclass(TypeError, Exception)

True

This also means that we can create our own exceptions that inherit from `Exception`. 

In [14]:
class KitchenException(Exception):
  """
  Exception that gets thrown when a kitchen appliance isn't working
  """

class MicrowaveException(KitchenException):
  """
  Exception for when the microwave stops working
  """

class RefrigeratorException(KitchenException):
  """
  Exception for when the refrigerator stops working
  """

We define `KitchenException` which inherits from the base, `Exception` class, so it inherits the behaviour of a regular **exception**. We then subclass `KitchenException` to create specific exceptions, each being raised under different circumstances. 

```py
def get_food_from_fridge():
  if refrigerator.cooling == False:
    raise RefrigeratorException
  else:
    return food

def heat_food(food):
  if microwave.working == False:
    raise MicrowaveException
  else:
    microwave.cook(food)
    return food

try:
  food = get_food_from_fridge() # potentially raises RefrigeratorException
  food = heat_food(food) # potentially raises MicrowaveException
except KitchenException: # catches EITHER
  food = order_takeout()
```

In the above example, we attempt to retrieve food from the fridge and heat it in the microwave. If either `RefrigeratorException` or `MicrowaveException` is raised, we opt to order takeout instead. We catch both exceptions in our `try/except` block because both are subclasses of `KitchenException`.

In [12]:
# Define a custom error that 
class OutOfStock(Exception):
    '''The current item is out of stock'''
    def __init__(self, message):
        self.message = message

# Update the class below to raise OutOfStock
class CandleShop:
    name = "Here's a Hot Tip: Buy Drip Candles"
    def __init__(self, stock):
        self.stock = stock
    
    def buy(self, color):
        if self.stock[color] < 1:
            raise OutOfStock('The requested item is out of stock')
        self.stock[color] -= 1

candle_shop = CandleShop({'blue': 6, 'red': 2, 'green': 1})
candle_shop.buy('blue')
candle_shop.buy('red')
candle_shop.buy('green')
print(candle_shop.stock)

{'blue': 5, 'red': 1, 'green': 0}


In [14]:
# This should raise OutOfStock:
try:
    candle_shop.buy('green')
except OutOfStock as error:
    print('OutOfStock:', error)

OutOfStock: The requested item is out of stock


### Overriding Methods

We can **override** methods in a subclass by simply providing a new definition of the method. When overriding methods the method name remains the same, but the parameters and code block can change.

In [36]:
class User:
    def __init__(self, username, permissions):
        self.username = username
        self.permissions = permissions

    def has_permission_for(self, key):
        if self.permissions.get(key):
            return True
        else:
            return False

class Admin(User):
    def has_permission_for(self, key):
        return True

Above we defined a class `User` which takes a parameter, permissions, which is a dictionary in its constructor. The method `.has_permission_for()` checks to see if a given key is in its permissions dictionary.

Here we define an `Admin` class that subclasses `User`. It has all methods, attributes, and functionality that `User` has. However, `has_permission_for()` does not check its permissions dictionary, it simply returns `True` everytime.

In [32]:
user = User('tom', {'file': True})
admin = Admin('mike', {'file': False})

In [33]:
user.has_permission_for('file')

True

In [34]:
admin.has_permission_for('file')

True

In some cases we do not want to re-define an existing method, but add some logic to the existing method. We can do so using `super()`. It provides a **proxy object** with which we can call the method in the parent/base/superclass. We call the required function as a method on `super()`.

In [35]:
class Sink:
    def __init__(self, basin, nozzle):
        self.basin = basin
        self.nozzle = nozzle

class KitchenSink(Sink):
    def __init__(self, basin, nozzle, trash_compactor=None):
        super().__init__(self, basin, nozzle)
        if trash_compactor:
            self.trash_compactor = trash_compactor

`KitchenSink`'s constructor takes an additional parameter, `trash_compactor`, and then calls the constructor for `Sink` with the `basin` and `nozzle` parameters it received using the `super()` function in the line `super().__init__(self, basin, nozzle)`.

In this way, we can override a parent class's method to add some new functionality (like adding a `trash_compactor` to a sink), while still retaining the behavior of the original constructor (like setting the `basin` and `nozzle` as instance variables).

In [48]:
class PotatoSalad:
    def __init__(self, potatoes, celery, onions):
        self.potatoes = potatoes
        self.celery = celery
        self.onions = onions
    
class PotatoSaladPlus(PotatoSalad):
    def __init__(self, potatoes, celery, onions):
        super().__init__(potatoes, celery, onions)
        self.raisins = 40
    
class PotatoSaladMinus(PotatoSalad):
    def __init__(self):
        self.potatoes = 5
        
saladPlus = PotatoSaladPlus(5, 20, 12)
saladMinus = PotatoSaladMinus()

### Interfaces

There are times when it might not matter which class an object is an instance of, all we're interested in is whether the object can perform a particular task(s), i.e. has a particular behaviour.

```py
class Chess:
  def __init__(self):
    self.board = setup_board()
    self.pieces = add_chess_pieces()

  def play(self):
    print("Playing chess!")

class Checkers:
  def __init__(self):
    self.board = setup_board()
    self.pieces = add_checkers_pieces()

  def play(self):
    print("Playing checkers!")
```

Both classes define the same constructor and have a `play()` method.  If we have a `play_game()` function that takes either object instance, it could call the `.play()` method without having to check which class the object is an instance of.

```py
def play_game(chess_or_checkers):
  chess_or_checkers.play()

chess_game = Chess()
checkers_game = Checkers()
chess_game_2 = Chess()

for game in [chess_game, checkers_game, chess_game_2]:
  play_game(game)

Playing chess!
Playing checkers!
Playing chess!
```

When two classes have the same method names and attributes, we say they implement the same interface - different objects from different classes can perform the same operation (even if it is implemented differently for each class).

### Polymorphism

Polymorphism is used to describe the same method carrying out different actions depending on the type of data, e.g `+` will add two numbers, will concatenating two strings and combining two lists. While `len()` will count the number of elements in a list, number of attributes in a dictionary and return the length of a string.

One way that we can introduce polymorphism to our class definitions is by using Python's special dunder methods. We ca use these so that our custom classes look and behave like built-in classes.

In [51]:
class Color:
    def __init__(self, red, blue, green):
        self.red = red
        self.blue = blue
        self.green = green

    def __repr__(self):
        return "Color with RGB = ({red}, {blue}, {green})".format(red=self.red, blue=self.blue, green=self.green)

    def __add__(self, other):
        """
        Adds two RGB colors together
        Maximum value is 255
        """
        new_red = min(self.red + other.red, 255)
        new_blue = min(self.blue + other.blue, 255)
        new_green = min(self.green + other.green, 255)

        return Color(new_red, new_blue, new_green)

red = Color(255, 0, 0)
blue = Color(0, 255, 0)
green = Color(0, 0, 255)

We can now add the colors together with the `+` operator.

In [52]:
# Color with RGB: (255, 255, 0)
magenta = red + blue

# Color with RGB: (0, 255, 255)
cyan = blue + green

# Color with RGB: (255, 0, 255)
yellow = red + green

# Color with RGB: (255, 255, 255)
white = red + blue + green

In [56]:
class Atom:
    def __init__(self, label):
        self.label = label
    
    def __add__(self, other):
        return Molecule([self, other])
    
class Molecule:
    def __init__(self, atoms):
        if type(atoms) is list:
            self.atoms = atoms
      
sodium = Atom("Na")
chlorine = Atom("Cl")
salt = sodium + chlorine
salt
# salt = sodium + chlorine

There are other Python **dunder** methods that allow us to use the same syntax as python's built in data types, e.g. we can **override** the `__iter__()`, `__len__()` and `__contains__()` methods so that our custom class behaves like a list. This allows you to use syntax you already know for built in classes on your custom classes.

In [57]:
class UserGroup:
    def __init__(self, users, permissions):
        self.user_list = users
        self.permissions = permissions

    def __iter__(self):
        return iter(self.user_list)

    def __len__(self):
        return len(self.user_list)

    def __contains__(self, user):
        return user in self.user_list

`__iter__` -  the iterator, we use the `iter()` function to turn the list `self.user_list` into an iterator so we can use `for user in user_group` syntax.

`__len__` - the length method, so when we call `len(user_group)` it will return the length of the underlying `self.user_list` list.

`__contains__` - the check for containment, allows us to use `user in user_group` syntax to check if a User exists in the `user_list` we have.

In [59]:
class User:
    def __init__(self, username):
        self.username = username

diana = User('diana')
frank = User('frank')
jenn = User('jenn')

can_edit = UserGroup([diana, frank], {'can_edit_page': True})
can_delete = UserGroup([diana, jenn], {'can_delete_posts': True})

In [62]:
# check the three methods
print(len(can_edit))
print(len(can_delete))

2
2


In [63]:
for user in can_edit:
    print(user.username)

diana
frank


In [64]:
if frank in can_delete:
    print('I found you!')
else:
    print('Where are you!')

Where are you!


In [67]:
class SortedList(list):
  # sort the list upon initialization
    def __init__(self, lst):
        super().__init__(lst)
        self.sort()
    
  # add an item to the list and sort it
    def append(self, value):
        super().append(value)
        self.sort()

In [69]:
sorted_list = SortedList([2,6,1,22,11,55,5,7,9,4,6,3,1,3,5,1,88,65])
sorted_list

[1, 1, 1, 2, 3, 3, 4, 5, 5, 6, 6, 7, 9, 11, 22, 55, 65, 88]

In [70]:
sorted_list.append(100)
sorted_list.append(63)
sorted_list.append(29)
sorted_list

[1, 1, 1, 2, 3, 3, 4, 5, 5, 6, 6, 7, 9, 11, 22, 29, 55, 63, 65, 88, 100]